In [2]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
! pip install fuzzywuzzy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import math
import os
from pandas.core.frame import DataFrame
import matplotlib.pyplot as plt
import numpy as np
import tkinter
import tkinter.messagebox
import matplotlib.pyplot as plt
import pandas as pd
from fuzzywuzzy import process

/usr/local/lib/python3.7/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Give the search text and the fuzzy searchtext, number

In [ ]:
search  ="FeCitrate"
fuzzy = "FeCitrate"
Number = 20

# Search function

In [ ]:
dirlast = "/content/gdrive/MyDrive/IRP"

def readDir(dirPath):
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        for f in fileList:
            if(f!="extraction" and f!=".DS_Store"):    
             f = dirPath+'/'+f
             allFiles.append(f)
    return allFiles 

# fix search Extract
def extract(text):
  # The path of the output files.
  tabel =dirlast+ '/tabel'+'/'+ text + 'fix.csv'
  logk = []
  reaction = []
  datafrom = []
  #Read files 
  for file in readDir(dirlast+"/data"):
    f = open(file,'r')
    # Save the dataset this search result from 
    dataset = file[file.find("data")+5:50].strip(".txt")
    check = 0
    lines = f.readlines()
    linenumber = 0
    # Begin the search
    for i in range(len(lines)-2):
      # First ensure the search text is in this line, and then ensure it is a reaction by "="
      if text in lines[i] and  "=" in lines[i]:
          # There is two format of logk, neighboor and two lines behind the last one.
          if "log_k"in lines[i+1]:
           linenumber = i+1
          if "log_k"in lines[i+2]:
           linenumber = i+2
          # Extract logK according to the different situation
          if "log_k"in lines[i+1] or "log_k"in lines[i+2]:
          # Get the reaction
           react = lines[i].strip().strip('#').strip()
           reaction.append(react)
           # Get logk
           k = lines[linenumber][lines[linenumber].find("k")+1:50].strip()
           # only save the logk when k not euaql to  0
           if k != 0:
            logk.append(k)
            datafrom.append(dataset)
            # Write the output file.
            
            # Save the data into dataframe 
            data = {"Reaction":reaction,"logk":logk,"dataset":datafrom}
            dataframe=DataFrame(data)
            # Save the dataframe to the csv file.
            dataframe.to_csv(tabel,index=False,sep=',')
  return dataframe

def fuzzyextract(text,number = 100):
  # The path of the output files.
  tabel =dirlast+ '/tabel'+'/'+ text + 'fuzzy.csv'
  # List conclude all logk value.
  logk = []
  # List conclude all reaction text.
  reaction = []
  # List conclude all dataset the reaction from.
  datafrom = []
  logknew = []
  reactionnew = []
  datafromnew = []
  name = []
  namenew = []
  
  for file in readDir(dirlast+"/data"):
    f = open(file,'r')
    dataset = file[file.find("data")+5:50].strip(".txt")
    check = 0
    ks = 0.02
    lines = f.readlines()
    checkifexist = False
    linenumber = 0
    for i in range(len(lines)-2):
      checks = False
      # Save all the reactions
      if  "=" in lines[i]:
         # if "E" in lines[i]:
          if "log_k"in lines[i+1]:
           linenumber = i+1
           checks = True
          if "log_k"in lines[i+2]:
           linenumber = i+2
           checks = True
          ks = lines[linenumber][lines[linenumber].find("k")+1:50].strip()
          
          if "#" in ks:
               ks = ks[0: ks.find("#")-2]
          if checks==True:
           
           react = lines[i].strip().strip('#').strip()
           named = lines[i-1].strip()
           reaction.append(react)
           name.append(named)
           k = ks
           logk.append(k)
           datafrom.append(dataset)
  # Do fuzzy serach to the all reactions to match the search text.
  m = process.extract(text, reaction, limit=number)  
  # Save the serach result.
  for j in range(len(reaction)):
      for best in m:
          if best[0] in reaction[j]:
            if check== 0:
              reactionnew.append(best[0])
              datafromnew.append(datafrom[j])
              logknew.append(logk[j])
              namenew.append(name[j])
              check = check+1
            else:
                for o in range(len(reactionnew)):
                    if best[0] in reactionnew[o] and logk[j] in logknew[o]:
                        checkifexist = True
                if checkifexist == False:
                       reactionnew.append(best[0])
                       datafromnew.append(datafrom[j])
                       logknew.append(logk[j])
                       namenew.append(name[j]) 
                checkifexist = False
               
  data = {"Reaction":reactionnew,"logk":logknew,"dataset":datafromnew,"name":namenew}
           
  dataframe=DataFrame(data)
        
  dataframe.to_csv(tabel,index=False,sep=',')      
  fileList = os.listdir(dirlast+ '/tabel')
  for fl in fileList:
     if os.path.getsize(dirlast+ '/tabel/'+fl) == 0:
                os.remove(dirlast+ '/tabel/'+fl)
                return False
  return dataframe

# Run code

Run the fuzzy seach code, the output file will in the tabel folder.

In [ ]:
print(fuzzyextract(fuzzy,Number))

                              Reaction   logk         dataset  \
0                              e- = e-    0.0  Tipping_Hurley   
1                              F- = F-    0.0  Tipping_Hurley   
2                              I- = I-    0.0  Tipping_Hurley   
3                              e- = e-  0.000             iso   
4                              F- = F-  0.000             iso   
5        Fe+2 + Citrate-3 = FeCitrate-    5.7          minteq   
6   Fe+2 + Citrate-3 + H+ = FeCitrateH    3.5          minteq   
7         Fe+3 + Citrate-3 = FeCitrate  12.55          minteq   
8  Fe+3 + Citrate-3 + H+ = FeCitrateH+   19.8          minteq   
9      Fe+2 + Citrate-3 = Fe(Citrate)-    6.1       minteq.v4   

                                 name  
0          #e- primary master species  
1          #F- primary master species  
2          #I- primary master species  
3                                      
4                                      
5                delta_h 0       kcal  
6   

Run the fix search code, the utput file will in the tabel folder

In [ ]:
print(extract(search))

                              Reaction   logk dataset
0        Fe+2 + Citrate-3 = FeCitrate-    5.7  minteq
1   Fe+2 + Citrate-3 + H+ = FeCitrateH    3.5  minteq
2         Fe+3 + Citrate-3 = FeCitrate  12.55  minteq
3  Fe+3 + Citrate-3 + H+ = FeCitrateH+   19.8  minteq
